# User data

This notebook looks at `dataset/user_data.csv` and prepares it for use. Saving the clean dataset in `dataset/user_data_cleaned`.

# Imports

In [17]:
%reset -f

import pandas as pd
import numpy as np
import random

# Prepare dataset

In [18]:
df = pd.read_csv(
    '../dataset/user_data.csv',
    header=None,
    names=["user_id", "game", "behavior", "value", "dummy"],
)

print(len(df))

200000


In [19]:
print(df.isnull().sum())

user_id     0
game        0
behavior    0
value       0
dummy       0
dtype: int64


In [20]:
user_games = df.groupby("user_id")["game"].nunique()
print("Average number of distinct games per user:", user_games.mean())
print("25th percentile:", user_games.quantile(0.25))
print("25th percentile:", user_games.quantile(0.5))
print("75th percentile:", user_games.quantile(0.75))

Average number of distinct games per user: 10.393286532720085
25th percentile: 1.0
25th percentile: 2.0
75th percentile: 6.0


In [6]:
popularity = df["game"].value_counts()
print("Most popular games:\n", popularity.head(10))

Most popular games:
 game
Dota 2                             9682
Team Fortress 2                    4646
Counter-Strike Global Offensive    2789
Unturned                           2632
Left 4 Dead 2                      1752
Counter-Strike Source              1693
Counter-Strike                     1424
Garry's Mod                        1397
The Elder Scrolls V Skyrim         1394
Warframe                           1271
Name: count, dtype: int64


In [7]:
unique_users = df["user_id"].nunique()
unique_games = df["game"].nunique()

print(f"Number of unique users: {unique_users}")
print(f"Number of unique games: {unique_games}")

Number of unique users: 12393
Number of unique games: 5155


In [8]:
df.drop(columns=["dummy"], inplace=True)

In [9]:
duplicates = df[df.duplicated(subset=["user_id", "game"], keep=False)]
print("These entries have duplicate (user_id, game) pairs:")
display(duplicates)

These entries have duplicate (user_id, game) pairs:


,user_id,game,behavior,value
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0
...,...,...,...,...
199995,128470551,Titan Souls,play,1.5
199996,128470551,Grand Theft Auto Vice City,purchase,1.0
199997,128470551,Grand Theft Auto Vice City,play,1.5
199998,128470551,RUSH,purchase,1.0


In [10]:
priority_map = {"play": 1, "purchase": 0}
df["priority"] = df["behavior"].map(priority_map)

df.sort_values(by=["user_id", "game", "priority"], 
               ascending=[True, True, False], 
               inplace=True)

df.drop_duplicates(subset=["user_id", "game"], keep="first", inplace=True)
df.drop(columns=["priority"], inplace=True)

print(len(df))

128804


In [11]:
df_names_appid = pd.read_csv(
    '../dataset/names_appid_map.csv',
)

In [12]:
import re
EMOJI_PATTERN = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "]+", flags=re.UNICODE)

def clean_game_name(name):
    name = name.lower()
    name = EMOJI_PATTERN.sub(r'', name)
    name = re.sub(r'[^a-z0-9 ]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name


In [13]:
df_names_appid = df_names_appid.rename(columns={"name": "clean_name"})
df["clean_name"] = df["game"].apply(clean_game_name)

df = df.merge(df_names_appid, on="clean_name", how="left")
df["appid"] = df["appid"].astype(pd.Int64Dtype(), errors="ignore")
df.drop(columns=["game", "clean_name"], inplace=True)

df.dropna(subset=["appid"], inplace=True)

In [14]:
print(df.head(10))

    user_id  behavior  value   appid
0      5250      play    4.9     630
1      5250      play  144.0  255710
2      5250  purchase    1.0      10
3      5250  purchase    1.0     240
4      5250  purchase    1.0      30
5      5250  purchase    1.0      40
7      5250      play    0.2     570
8      5250  purchase    1.0      70
9      5250  purchase    1.0     220
10     5250  purchase    1.0     320


In [16]:
print(len(df))

98048


In [79]:
df.to_csv("../dataset/user_data_cleaned.csv", index=False)